In [ ]:
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from typing import Dict, Any, List
from dotenv import load_dotenv

api_key = os.getenv("GEMINI_API_KEY")

In [ ]:
MAX_DEPTH = 4
ITEMS_PER_LEVEL = 5

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=api_key,
    temperature=0,
)
# ここでは、親となるトピックと、そこに至るまでのパスを合わせて提供します。
PROMPT_TEMPLATE = ChatPromptTemplate.from_messages([
    ("system", "あなたはマインドマップの項目生成を支援するアシスタントです。指定された親トピックに対する、次の階層の主要な副トピックや関連概念を{items_per_level}個ほどリストアップしてください。全体像との関連性を保つために、マインドマップ全体のターゲットと、現在の位置を示すパス情報が与えられます。リスト形式（'- 項目名' の形式）で、項目名のみを出力してください。他の余分なテキストは含めないでください。"),
    ("human", """
マインドマップ全体のターゲット: {root_topic}
現在の位置パス: {context_path}
次の階層を生成する親トピック: {parent_topic}
""")
])

# プロンプトとLLMをチェーンとして結合
chain = PROMPT_TEMPLATE | llm

In [ ]:
# マインドマップを格納するための再帰的な辞書構造
MindMapNode = Dict[str, Any] # 例: {"項目名": {"子項目名1": {...}, "子項目名2": {...}}}

In [ ]:
# --- マインドマップ生成関数（再帰・文脈考慮） ---
# path_list: ルートから現在のparent_itemまでの項目名のリスト 例: ["ターゲット", "第1層項目A", "第2層項目A-1"]
def generate_layer(parent_item: str, parent_dict: MindMapNode, current_depth: int, path_list: List[str]):
    """
    指定された親項目に対して、次の階層の項目をLLMに生成させ、
    再帰的にマインドマップの階層を構築する。文脈として親までのパスを考慮する。

    Args:
        parent_item (str): 現在の親となる項目名。
        parent_dict (MindMapNode): 親項目の子ノードを格納するための辞書オブジェクト。
        current_depth (int): 現在生成しようとしている階層の深さ (ルートを0として数える)。
                             ここで生成するのは current_depth の項目です。
        path_list (List[str]): ルートから parent_item までの項目名のリスト。
    """
    # 指定された最大深度に達したら、このブランチの生成を終了
    if current_depth >= MAX_DEPTH:
         return

    # Notebookのセル出力として進捗を表示
    # パスを ">" で繋いで表示すると分かりやすい
    path_str_for_display = " > ".join(path_list)
    print(f"  {'  ' * (current_depth - 1)}➡️ Layer {current_depth} を生成中（パス: {path_str_for_display}）") # 進捗表示

    try:
        # LLMに子項目を生成させる際に、ルートトピックと現在のパスを提供する
        root_topic = path_list[0] # パスの最初の要素がルートトピック
        context_path_str = " > ".join(path_list) # パスを文字列に変換

        response = chain.invoke({
            "root_topic": root_topic,
            "context_path": context_path_str,
            "parent_topic": parent_item,
            "items_per_level": ITEMS_PER_LEVEL
        })

        # LLMの応答から項目リストをパース
        raw_items = response.content.strip().split('\n')

        child_items = []
        for item in raw_items:
            cleaned_item = item.strip()
            if cleaned_item.startswith('- '):
                cleaned_item = cleaned_item[2:].strip() # '- ' を除去
            if cleaned_item: # 空でない項目のみ追加
                child_items.append(cleaned_item)

        # 生成された子項目がない場合は、このブランチはここで終了
        if not child_items:
            # Notebookのセル出力として表示
            # print(f"  {'  ' * (current_depth - 1)}    └ 子項目が生成されませんでした。")
            return # 子項目がなければ再帰は終了

        # 生成された各子項目に対して再帰的に処理を行う
        for child_item in child_items:
            # 子項目を親辞書に追加し、その子項目を格納するための空の辞書を値とする
            parent_dict[child_item] = {}
            # 次の階層 (current_depth + 1) の項目を生成するために再帰呼び出し
            # パスリストに現在の子項目を追加して渡す
            generate_layer(child_item, parent_dict[child_item], current_depth + 1, path_list + [child_item])

    except Exception as e:
        # Notebookのセル出力として表示
        print(f"\nエラー発生: '{parent_item}' の子項目生成中にエラーが発生しました: {e}")
        print(f"パス: {path_str_for_display}")
        print(f"このブランチの生成を中断します。\n")
        # エラー発生時はそのブランチの生成を中断する

In [ ]:
# --- Markdown形式で出力するための関数（再帰） ---
def format_mind_map_markdown(mind_map_dict: MindMapNode, indent_level: int = 0) -> str:
    """
    マインドマップの辞書構造をMarkdown形式の文字列に変換する。

    Args:
        mind_map_dict (MindMapNode): 現在処理中のマインドマップの辞書（部分）。
        indent_level (int): 現在の階層のインデントレベル (0から開始)。

    Returns:
        str: 生成されたMarkdown形式の文字列。
    """
    markdown_output = ""
    # 階層に応じてインデント文字列を生成 (Markdownではスペース2つが一般的)
    indent = "  " * indent_level

    # 辞書の各項目 (項目名とその子項目) を処理
    for item, children in mind_map_dict.items():
        # 現在の項目をリスト形式で追加 (インデントと '- ')
        markdown_output += f"{indent}- {item}\n"
        # 子項目がある場合、再帰的にフォーマット関数を呼び出し、結果を追加
        if children:
            markdown_output += format_mind_map_markdown(children, indent_level + 1)

    return markdown_output

print("LLMセットアップ、文脈考慮版生成関数、フォーマット関数の定義が完了しました。")

In [ ]:
# マインドマップを作成したいターゲットをここに直接入力してください
target_topic = "タンタル導電性高分子コンデンサのリフロークラックに影響する要因"

print(f"ターゲットトピックを '{target_topic}' に設定しました。次のセルで生成を開始します。")

In [ ]:
print(f"'{target_topic}' のマインドマップを作成します（最大{MAX_DEPTH}層）...")
print("生成には時間がかかる場合があります。\n")

# マインドマップのデータ構造の初期化
mind_map_data: MindMapNode = {target_topic: {}}

# マインドマップの生成を開始
# generate_layer関数の初期呼び出しでは、
# parent_item = target_topic
# parent_dict = mind_map_data[target_topic] （target_topicの子を格納する辞書）
# current_depth = 1 （Layer 1 をこれから生成する）
# path_list = [target_topic] （現在の項目(parent_item)までのパス、ここではターゲット自身のみ）
generate_layer(target_topic, mind_map_data[target_topic], 1, [target_topic])

print("\n--- マインドマップ生成完了 ---")

In [ ]:
# Jupyter環境の場合、IPythonのdisplay関数を使ってMarkdownをレンダリングします
from IPython.display import Markdown, display


print("--- マインドマップ (Markdown形式) ---")

# ターゲット（ルート）をMarkdownのH1見出しとして含む文字列を作成
markdown_output_string = f"# {target_topic}\n\n" # 見出しとリストの間に空行を追加

# 生成されたマインドマップデータをMarkdown形式に変換し、文字列に追加
# target_topic の子項目から開始するため、mind_map_data[target_topic] を渡す
# 最初の階層 (Layer 1) はインデントレベル0から開始する
markdown_output_string += format_mind_map_markdown(mind_map_data[target_topic], indent_level=0)

# IPython.display.Markdown を使用して、Notebookの出力としてMarkdownをレンダリング
display(Markdown(markdown_output_string))

print("----------------------------------")